In [ ]:
# ------- PARAMETERS ------- # 

RUNS  = [205,207,208,210,212,213,214,215,216,217,221,222,223,224,225,226]
RATES = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

root = '/pnfs/pic.es/data/cta/LST/LST2/Data/ZFITS/'

# -------------------------- #

# other parameters
output_path = 'file_output/'
onlySubrun0 = True              # if we only extract the data for the subrun 0

In [ ]:
import numpy  as np
import pandas as pd
import os

from traitlets.config      import Config
from ctapipe.io            import EventSource
from IPython.display       import clear_output

import auxiliar as aux

# extracting the number of LST
LST_camera = aux.find_LST_num(root)

# creating the folder if dont exist
aux.create_folder(output_path) 

In [ ]:
# over all runs
for run in RUNS:

    # finding the run in the folder root
    date, subruns = aux.search(root, run)    

    # to open a second run we need to use the first subrun of the run and
    # extract the timestamp of the first event

    # first subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,
            },
        })  

    path   = root+date+'/'
    path   = path + 'LST-' + LST_camera + '.1.Run' + str(run).zfill(5) + '.' + str(0).zfill(4) + '.fits.fz'
    source = EventSource(input_url=path, config=config, max_events=1)

    for i, ev in enumerate(source): 
        timestamp = int(ev.trigger.time.value)

    # following subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,

          "EventTimeCalculator": {
            "dragon_module_id":133 ,
            "dragon_reference_counter": 0,
            "dragon_reference_time": timestamp,
          },
            },
        })
    
    # over all subruns
    if onlySubrun0 == True:
        subruns = [subruns[0]]
        
    for subrun in subruns:
        
        print('Creating Run '+str(run)+' Subrun '+str(subrun) + '...\n') 
        
        # directory
        path   = root + date + '/'
        path   = path + 'LST-2.1.Run' + str(run).zfill(5)+'.'+str(subrun).zfill(4)+'.fits.fz'
        source = EventSource(input_url=path, config=config)


        # arrays 
        time = []

        for i, ev in enumerate(source): 

            time.append(ev.trigger.time.value)
            
        print('\n Creating the dataframe...')
        
        #we create the matrix for the dataframe
        matrix = [time]  

        del time

        # creating the dataframe using pandas
        data = pd.DataFrame(np.transpose(matrix))

        del matrix

        # csv arxive
        data.to_csv(output_path + 'data_tRes_Run' + str(run) + '_Subrun' + str(subrun) +
                    '_Rate' + str(RATES[RUNS.index(run)]) + '.csv', index=False)

        # clear memory and console
        clear_output()
        del data
        
        print('Finished Run '+str(run)+' Subrun '+str(subrun) + '\n')